# Stacking Ensemble with Out-of-Fold Predictions

## Goal: Push toward 0.80+ LB using proven stacking approach

Reference: arthurtok kernel achieved 0.808 LB (top 9%) with stacking

Current best: LB 0.7847 (Threshold-Tuned Ensemble)

Approach:
1. 5 base models: RF, ExtraTrees, AdaBoost, GradientBoosting, SVC
2. Generate OOF predictions (avoid leakage)
3. XGBoost meta-learner on stacked predictions
4. Apply threshold tuning for ~31% survival rate

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import xgboost as xgb
import warnings
warnings.filterwarnings('ignore')

# Load data
train = pd.read_csv('/home/data/train.csv')
test = pd.read_csv('/home/data/test.csv')

print(f"Train shape: {train.shape}")
print(f"Test shape: {test.shape}")

Train shape: (891, 12)
Test shape: (418, 11)


In [2]:
# Preprocessing (same as previous experiments)
def preprocess_with_title(train_df, test_df):
    train_data = train_df.copy()
    test_data = test_df.copy()
    
    # Title extraction
    for df in [train_data, test_data]:
        df['Title'] = df['Name'].str.extract(r' ([A-Za-z]+)\.', expand=False)
        title_mapping = {
            'Mr': 'Mr', 'Miss': 'Miss', 'Mrs': 'Mrs', 'Master': 'Master',
            'Mlle': 'Miss', 'Ms': 'Miss', 'Mme': 'Mrs',
            'Lady': 'Rare', 'Countess': 'Rare', 'Capt': 'Rare', 'Col': 'Rare',
            'Don': 'Rare', 'Dr': 'Rare', 'Major': 'Rare', 'Rev': 'Rare',
            'Sir': 'Rare', 'Jonkheer': 'Rare', 'Dona': 'Rare'
        }
        df['Title'] = df['Title'].map(title_mapping).fillna('Rare')
    
    # Encodings
    title_order = ['Mr', 'Miss', 'Mrs', 'Master', 'Rare']
    title_map = {t: i for i, t in enumerate(title_order)}
    train_data['Title_Code'] = train_data['Title'].map(title_map)
    test_data['Title_Code'] = test_data['Title'].map(title_map)
    
    train_data['Sex_Code'] = (train_data['Sex'] == 'male').astype(int)
    test_data['Sex_Code'] = (test_data['Sex'] == 'male').astype(int)
    
    train_data['Embarked'] = train_data['Embarked'].fillna('S')
    test_data['Embarked'] = test_data['Embarked'].fillna('S')
    embarked_map = {'S': 0, 'C': 1, 'Q': 2}
    train_data['Embarked_Code'] = train_data['Embarked'].map(embarked_map)
    test_data['Embarked_Code'] = test_data['Embarked'].map(embarked_map)
    
    # Fare and Age imputation from train only
    train_fare_median = train_data['Fare'].median()
    train_data['Fare'] = train_data['Fare'].fillna(train_fare_median)
    test_data['Fare'] = test_data['Fare'].fillna(train_fare_median)
    
    age_medians = train_data.groupby(['Title', 'Pclass'])['Age'].median()
    train_age_median = train_data['Age'].median()
    
    def fill_age(row, medians, fallback):
        if pd.isna(row['Age']):
            try:
                return medians[(row['Title'], row['Pclass'])]
            except KeyError:
                return fallback
        return row['Age']
    
    train_data['Age'] = train_data.apply(lambda x: fill_age(x, age_medians, train_age_median), axis=1)
    test_data['Age'] = test_data.apply(lambda x: fill_age(x, age_medians, train_age_median), axis=1)
    
    return train_data, test_data

train_processed, test_processed = preprocess_with_title(train, test)

In [3]:
# Prepare data
feature_cols = ['Pclass', 'Sex_Code', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked_Code', 'Title_Code']

X = train_processed[feature_cols].values
y = train_processed['Survived'].values
X_test = test_processed[feature_cols].values
test_ids = test_processed['PassengerId'].values

print(f"Features: {feature_cols}")
print(f"X shape: {X.shape}")
print(f"X_test shape: {X_test.shape}")

Features: ['Pclass', 'Sex_Code', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked_Code', 'Title_Code']
X shape: (891, 8)
X_test shape: (418, 8)


In [4]:
# Define OOF prediction function (CRITICAL: avoid leakage)
def get_oof_predictions(clf, X, y, X_test, n_splits=5, scale=False):
    """
    Generate out-of-fold predictions for stacking.
    Returns OOF predictions for train and averaged predictions for test.
    """
    kfold = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
    
    oof_train = np.zeros((len(X),))
    oof_test = np.zeros((len(X_test),))
    oof_test_folds = np.zeros((n_splits, len(X_test)))
    
    for i, (train_idx, val_idx) in enumerate(kfold.split(X, y)):
        X_train_fold, X_val_fold = X[train_idx], X[val_idx]
        y_train_fold = y[train_idx]
        X_test_fold = X_test.copy()
        
        # Scale if needed (for SVC)
        if scale:
            scaler = StandardScaler()
            X_train_fold = scaler.fit_transform(X_train_fold)
            X_val_fold = scaler.transform(X_val_fold)
            X_test_fold = scaler.transform(X_test_fold)
        
        # Clone and fit
        clf_clone = clf.__class__(**clf.get_params())
        clf_clone.fit(X_train_fold, y_train_fold)
        
        # Predict
        oof_train[val_idx] = clf_clone.predict(X_val_fold)
        oof_test_folds[i, :] = clf_clone.predict(X_test_fold)
    
    # Average test predictions across folds
    oof_test = oof_test_folds.mean(axis=0)
    
    return oof_train.reshape(-1, 1), oof_test.reshape(-1, 1)

print("OOF prediction function defined")

OOF prediction function defined


In [5]:
# Define base models (parameters from reference kernel)
rf = RandomForestClassifier(
    n_estimators=500, 
    max_depth=6, 
    min_samples_leaf=2,
    random_state=42,
    n_jobs=-1
)

et = ExtraTreesClassifier(
    n_estimators=500,
    max_depth=8,
    min_samples_leaf=2,
    random_state=42,
    n_jobs=-1
)

ada = AdaBoostClassifier(
    n_estimators=500,
    learning_rate=0.75,
    random_state=42
)

gb = GradientBoostingClassifier(
    n_estimators=500,
    max_depth=5,
    min_samples_leaf=2,
    random_state=42
)

svc = SVC(
    kernel='linear',
    C=0.025,
    random_state=42
)

print("Base models defined:")
print("- RandomForest (n=500, depth=6)")
print("- ExtraTrees (n=500, depth=8)")
print("- AdaBoost (n=500, lr=0.75)")
print("- GradientBoosting (n=500, depth=5)")
print("- SVC (linear, C=0.025)")

Base models defined:
- RandomForest (n=500, depth=6)
- ExtraTrees (n=500, depth=8)
- AdaBoost (n=500, lr=0.75)
- GradientBoosting (n=500, depth=5)
- SVC (linear, C=0.025)


In [6]:
# Generate OOF predictions for each base model
print("Generating OOF predictions...")
print("="*50)

# RandomForest
print("Training RandomForest...")
rf_oof_train, rf_oof_test = get_oof_predictions(rf, X, y, X_test, scale=False)
rf_acc = accuracy_score(y, rf_oof_train.ravel())
print(f"  RF OOF Accuracy: {rf_acc:.4f}")

# ExtraTrees
print("Training ExtraTrees...")
et_oof_train, et_oof_test = get_oof_predictions(et, X, y, X_test, scale=False)
et_acc = accuracy_score(y, et_oof_train.ravel())
print(f"  ET OOF Accuracy: {et_acc:.4f}")

# AdaBoost
print("Training AdaBoost...")
ada_oof_train, ada_oof_test = get_oof_predictions(ada, X, y, X_test, scale=False)
ada_acc = accuracy_score(y, ada_oof_train.ravel())
print(f"  Ada OOF Accuracy: {ada_acc:.4f}")

# GradientBoosting
print("Training GradientBoosting...")
gb_oof_train, gb_oof_test = get_oof_predictions(gb, X, y, X_test, scale=False)
gb_acc = accuracy_score(y, gb_oof_train.ravel())
print(f"  GB OOF Accuracy: {gb_acc:.4f}")

# SVC (needs scaling)
print("Training SVC...")
svc_oof_train, svc_oof_test = get_oof_predictions(svc, X, y, X_test, scale=True)
svc_acc = accuracy_score(y, svc_oof_train.ravel())
print(f"  SVC OOF Accuracy: {svc_acc:.4f}")

print("\nAll base models trained!")

Generating OOF predictions...
Training RandomForest...


  RF OOF Accuracy: 0.8339
Training ExtraTrees...


  ET OOF Accuracy: 0.8238
Training AdaBoost...


  Ada OOF Accuracy: 0.8204
Training GradientBoosting...


  GB OOF Accuracy: 0.8159
Training SVC...
  SVC OOF Accuracy: 0.7957

All base models trained!


In [7]:
# Stack OOF predictions
X_train_stack = np.concatenate((rf_oof_train, et_oof_train, ada_oof_train, gb_oof_train, svc_oof_train), axis=1)
X_test_stack = np.concatenate((rf_oof_test, et_oof_test, ada_oof_test, gb_oof_test, svc_oof_test), axis=1)

print(f"Stacked train shape: {X_train_stack.shape}")
print(f"Stacked test shape: {X_test_stack.shape}")
print(f"\nStacked features: [RF, ET, Ada, GB, SVC]")

Stacked train shape: (891, 5)
Stacked test shape: (418, 5)

Stacked features: [RF, ET, Ada, GB, SVC]


In [8]:
# Train XGBoost meta-learner with CV
print("\nTraining XGBoost meta-learner...")
print("="*50)

kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# XGBoost parameters from reference kernel
xgb_params = {
    'n_estimators': 2000,
    'max_depth': 4,
    'learning_rate': 0.02,
    'gamma': 0.9,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'random_state': 42,
    'eval_metric': 'logloss',
    'use_label_encoder': False
}

oof_meta_preds = np.zeros(len(X_train_stack))
oof_meta_probs = np.zeros(len(X_train_stack))
test_meta_probs = np.zeros(len(X_test_stack))
fold_scores = []

for fold, (train_idx, val_idx) in enumerate(kfold.split(X_train_stack, y)):
    X_train_fold = X_train_stack[train_idx]
    X_val_fold = X_train_stack[val_idx]
    y_train_fold = y[train_idx]
    y_val_fold = y[val_idx]
    
    meta_model = xgb.XGBClassifier(**xgb_params)
    meta_model.fit(X_train_fold, y_train_fold, verbose=False)
    
    val_pred = meta_model.predict(X_val_fold)
    val_prob = meta_model.predict_proba(X_val_fold)[:, 1]
    
    oof_meta_preds[val_idx] = val_pred
    oof_meta_probs[val_idx] = val_prob
    test_meta_probs += meta_model.predict_proba(X_test_stack)[:, 1] / kfold.n_splits
    
    fold_acc = accuracy_score(y_val_fold, val_pred)
    fold_scores.append(fold_acc)
    print(f"Fold {fold+1}: {fold_acc:.4f}")

cv_score = accuracy_score(y, oof_meta_preds)
print(f"\nMeta-learner CV Accuracy: {cv_score:.4f} (+/- {np.std(fold_scores):.4f})")


Training XGBoost meta-learner...


Fold 1: 0.8603


Fold 2: 0.8371


Fold 3: 0.8427


Fold 4: 0.8371


Fold 5: 0.8708

Meta-learner CV Accuracy: 0.8496 (+/- 0.0136)


In [9]:
# Threshold analysis for stacking predictions
print("\n" + "="*60)
print("THRESHOLD ANALYSIS FOR STACKING")
print("="*60)

thresholds = [0.45, 0.50, 0.52, 0.55, 0.58, 0.60, 0.62, 0.65]

print(f"\n{'Threshold':<12} {'Survivors':<12} {'Survival Rate':<15} {'OOF Accuracy':<15}")
print("-"*55)

for thresh in thresholds:
    test_preds = (test_meta_probs >= thresh).astype(int)
    oof_preds = (oof_meta_probs >= thresh).astype(int)
    survivors = test_preds.sum()
    survival_rate = test_preds.mean()
    oof_acc = accuracy_score(y, oof_preds)
    
    marker = ""
    if 125 <= survivors <= 135:
        marker = " <- TARGET"
    
    print(f"{thresh:<12.2f} {survivors:<12} {survival_rate:<15.3f} {oof_acc:<15.4f}{marker}")


THRESHOLD ANALYSIS FOR STACKING

Threshold    Survivors    Survival Rate   OOF Accuracy   
-------------------------------------------------------
0.45         133          0.318           0.8496          <- TARGET
0.50         133          0.318           0.8496          <- TARGET
0.52         133          0.318           0.8485          <- TARGET
0.55         133          0.318           0.8451          <- TARGET
0.58         133          0.318           0.8384          <- TARGET
0.60         132          0.316           0.8384          <- TARGET
0.62         129          0.309           0.8361          <- TARGET
0.65         118          0.282           0.8339         


In [10]:
# Find optimal threshold for ~31% survival rate
target_survivors = 130

low, high = 0.4, 0.7
while high - low > 0.001:
    mid = (low + high) / 2
    survivors = (test_meta_probs >= mid).sum()
    if survivors > target_survivors:
        low = mid
    else:
        high = mid

optimal_threshold = (low + high) / 2
optimal_survivors = (test_meta_probs >= optimal_threshold).sum()

print(f"\nOptimal threshold for ~{target_survivors} survivors: {optimal_threshold:.3f}")
print(f"Actual survivors: {optimal_survivors}")
print(f"Survival rate: {optimal_survivors/len(test_meta_probs):.3f}")


Optimal threshold for ~130 survivors: 0.615
Actual survivors: 131
Survival rate: 0.313


In [11]:
# Create submission with optimal threshold
test_preds_final = (test_meta_probs >= optimal_threshold).astype(int)

submission = pd.DataFrame({
    'PassengerId': test_ids,
    'Survived': test_preds_final
})

submission.to_csv('/home/submission/submission.csv', index=False)
print(f"Submission saved with {len(submission)} rows")
print(f"\nSurvived distribution:")
print(submission['Survived'].value_counts())
print(f"\nSurvival rate: {submission['Survived'].mean():.3f}")

Submission saved with 418 rows

Survived distribution:
Survived
0    287
1    131
Name: count, dtype: int64

Survival rate: 0.313


In [12]:
# Summary
print("\n" + "="*70)
print("STACKING EXPERIMENT SUMMARY")
print("="*70)
print(f"\nBase Models OOF Accuracy:")
print(f"  RandomForest:     {rf_acc:.4f}")
print(f"  ExtraTrees:       {et_acc:.4f}")
print(f"  AdaBoost:         {ada_acc:.4f}")
print(f"  GradientBoosting: {gb_acc:.4f}")
print(f"  SVC:              {svc_acc:.4f}")

print(f"\nMeta-learner (XGBoost):")
print(f"  CV Accuracy (threshold 0.5): {cv_score:.4f}")
oof_acc_opt = accuracy_score(y, (oof_meta_probs >= optimal_threshold).astype(int))
print(f"  CV Accuracy (threshold {optimal_threshold:.3f}): {oof_acc_opt:.4f}")

print(f"\nSubmission:")
print(f"  Threshold: {optimal_threshold:.3f}")
print(f"  Survivors: {submission['Survived'].sum()} ({submission['Survived'].mean()*100:.1f}%)")

print(f"\nComparison to previous best:")
print(f"  Threshold-Tuned Ensemble (LB 0.7847): 130 survivors (31.1%)")
print(f"  This stacking model: {submission['Survived'].sum()} survivors ({submission['Survived'].mean()*100:.1f}%)")
print(f"\nExpected LB: ~0.78-0.81 (based on kernel reference achieving 0.808)")


STACKING EXPERIMENT SUMMARY

Base Models OOF Accuracy:
  RandomForest:     0.8339
  ExtraTrees:       0.8238
  AdaBoost:         0.8204
  GradientBoosting: 0.8159
  SVC:              0.7957

Meta-learner (XGBoost):
  CV Accuracy (threshold 0.5): 0.8496
  CV Accuracy (threshold 0.615): 0.8373

Submission:
  Threshold: 0.615
  Survivors: 131 (31.3%)

Comparison to previous best:
  Threshold-Tuned Ensemble (LB 0.7847): 130 survivors (31.1%)
  This stacking model: 131 survivors (31.3%)

Expected LB: ~0.78-0.81 (based on kernel reference achieving 0.808)
